In [26]:
FN = 'train'

you should use GPU but if it is busy then you always can fall back to your CPU

In [27]:
import os
# os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

In [28]:
import keras
keras.__version__

'1.2.2'

Use indexing of tokens from [vocabulary-embedding](./vocabulary-embedding.ipynb) this does not clip the indexes of the words to `vocab_size`.

Use the index of outside words to replace them with several `oov` words (`oov` , `oov0`, `oov1`, ...) that appear in the same description and headline. This will allow headline generator to replace the oov with the same word in the description

In [29]:
FN0 = 'vocabulary-embedding'

implement the "simple" model from http://arxiv.org/pdf/1512.01712v1.pdf

you can start training from a pre-existing model. This allows you to run this notebooks many times, each time using different parameters and passing the end result of one run to be the input of the next.

I've started with `maxlend=0` (see below) in which the description was ignored. I then moved to start with a high `LR` and the manually lowering it. I also started with `nflips=0` in which the original headlines is used as-is and slowely moved to `12` in which half the input headline was fliped with the predictions made by the model (the paper used fixed 10%)

In [30]:
FN1 = 'train'

input data (`X`) is made from `maxlend` description words followed by `eos`
followed by headline words followed by `eos`
if description is shorter than `maxlend` it will be left padded with `empty`
if entire data is longer than `maxlen` it will be clipped and if it is shorter it will be right padded with empty.

labels (`Y`) are the headline words followed by `eos` and clipped or padded to `maxlenh`

In other words the input is made from a `maxlend` half in which the description is padded from the left
and a `maxlenh` half in which `eos` is followed by a headline followed by another `eos` if there is enough space.

The labels match only the second half and 
the first label matches the `eos` at the start of the second half (following the description in the first half)

In [31]:
maxlend=25 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

the out of the first `activation_rnn_size` nodes from the top LSTM layer will be used for activation and the rest will be used to select predicted word

In [32]:
activation_rnn_size = 40 if maxlend else 0

In [33]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [34]:
#nb_train_samples = 30000
#nb_val_samples = 3000
nb_train_samples = 700
nb_val_samples = 299

# read word embedding

In [35]:
#import cPickle as pickle
import pickle

with open('data/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)
vocab_size, embedding_size = embedding.shape

In [36]:
with open('data/%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [37]:
nb_unknown_words = 10

In [38]:
print('number of examples',len(X),len(Y))
print('dimension of embedding space for words',embedding_size)
print('vocabulary size', vocab_size, 'the last %d words can be used as place holders for unknown/oov words'%nb_unknown_words)
print('total number of different words',len(idx2word), len(word2idx))
print('number of words outside vocabulary which we can substitue using glove similarity', len(glove_idx2idx))
print('numberof wordsthatwillbe regardedas unknonw(unk)/out-of-vocabulary(oov)',len(idx2word)-vocab_size-len(glove_idx2idx))

number of examples 999 999
dimension of embedding space for words 100
vocabulary size 40000 the last 10 words can be used as place holders for unknown/oov words
total number of different words 62811 62811
number of words outside vocabulary which we can substitue using glove similarity 6584
numberof wordsthatwillbe regardedas unknonw(unk)/out-of-vocabulary(oov) 16227


In [39]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

when printing mark words outside vocabulary with `^` at their end

In [40]:
oov0 = vocab_size-nb_unknown_words

In [41]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [43]:
print(nb_val_samples)
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

299


(700, 700, 299, 299)

In [44]:
del X
del Y

In [45]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [46]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

In [47]:
def prt(label, x):
    print(label+':',)
    for w in x:
        print(idx2word[w],)
    print()

In [48]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H:
Minor
Hotel
Group
to
co-invest
with
Nakheel
for
first
AVANI
Hotel
in
Dubai

D:
Thailand-based^
Minor
Hotel
Group
(MHG),
has
announced
a
strategic
partnership
with
Dubai-based
Nakheel
Group
to
co-invest
and
develop
a
500-key
Avani
hotel
in
Dubai,
to
be
opened
in
2018.
Avani
Deira
Dubai
Hotel
will
be
located
in
Deira
Islands,
a
unique
waterfront
entertainment,
tourist
and
residential
destination
being
developed
in
alignment
with
Dubai
Vision
2020.
Comprising^
four
islands
situated
adjacent
to
Dubai’s
Deira
district,
with
a
total
area
of
over
15.3
square
kilometres,^
Deira
Islands,
developed
by
Nakheel,
will
feature
a
variety
of
beachfront
resorts,^
hotels
and
residential
communities,
a
shopping
mall,
a
waterfront
night
market
and
an
impressive
amphitheatre^
able
to
accommodate
up
to
30,000
people.
The
Avani
hotel
will
be
close
to
Dubai
International
Airport,
with
views
of
the
Dubai
skyline^
and
across
the
Arabian
Gulf.^
The
new-build^
hotel
is
the
first
of
MHG’s
vibrant
Avani
brand
to

In [50]:
#i = 334
i = 100
prt('H',Y_test[i])
prt('D',X_test[i])

H:
Why
Are
the
Brightest^
Stars
on
Broadway
Sharing
Their
Secrets
of
Success
With
This
15-Year-Old^
Local
Greenville
Podcaster?^

D:
GREENVILLE,
S.C.
,
Sept.
17,
2015
/PRNewswire/
--
Many
young
performers
have
dreams
of
making
it
to
the
bright
lights
of
Broadway
someday,
but
few
have
taken
destiny^
into
their
own
hands
like
15-year-old
Eryn
Woo
of
Greenville,^
South
Carolina
.
Woo
interviews
actors,
directors,
playwrights,^
talent
agents
and
others
from
the
industry
on
her
podcast
at
http://www.goingforbroadway.com
.
Photo
-
http://photos.prnewswire.com/prnh/20150916/267181
Photo
-
http://photos.prnewswire.com/prnh/20150916/267453^
Woo
asks
insightful
questions
about
all
aspects
of
each
guest's
story.
She
delves^
into
the
details
of
the
chain
of
events
that
led
them
to
their
successful
debuts
on
Broadway
as
well
as
the
day-to-day
rituals
that
help
keep
them
stage-ready.^
Woo's
determination
to
gain
a
deep
understanding
of
her
guests
is
showcased
in
the
thoughtfulness
of
her
questions.


# Model

In [51]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector, Merge
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

In [52]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [53]:
regularizer = l2(weight_decay) if weight_decay else None

start with a standard stacked LSTM

In [54]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    W_regularizer=regularizer, dropout=p_emb, weights=[embedding], mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True, # batch_norm=batch_norm,
                W_regularizer=regularizer, U_regularizer=regularizer,
                b_regularizer=regularizer, dropout_W=p_W, dropout_U=p_U,
                name='lstm_%d'%(i+1)
                  )
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

A special layer that reduces the input just to its headline part (second half).
For each word in this part it concatenate the output of the previous layer (RNN)
with a weighted average of the outputs of the description part.
In this only the last `rnn_size - activation_rnn_size` are used from each output.
The first `activation_rnn_size` output is used to computer the weights for the averaging.

In [55]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [56]:
if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,
                                W_regularizer=regularizer, b_regularizer=regularizer,
                                name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

In [57]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [32]:
%%javascript
// new Audio("http://www.soundjay.com/button/beep-09.wav").play ()

<IPython.core.display.Javascript object>

In [58]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [59]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print(i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print(str_shape(weight),)
        print()

In [60]:
inspect_model(model)

0 cls=Embedding name=embedding_1
40000x100

1 cls=LSTM name=lstm_1
100x512
512x512
512
100x512
512x512
512
100x512
512x512
512
100x512
512x512
512

2 cls=Dropout name=dropout_1

3 cls=LSTM name=lstm_2
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512

4 cls=Dropout name=dropout_2

5 cls=LSTM name=lstm_3
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512
512x512
512x512
512

6 cls=Dropout name=dropout_3

7 cls=SimpleContext name=simplecontext_1

8 cls=TimeDistributed name=timedistributed_1
944x40000
40000

9 cls=Activation name=activation_1



# Load

In [61]:
if FN1 and os.path.exists('data/%s.hdf5'%FN1):
    model.load_weights('data/%s.hdf5'%FN1)

# Test

In [62]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [63]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [64]:
np.all(data[:,maxlend] == eos)

True

In [65]:
data.shape,map(len, samples)

((1, 50), <map at 0x7f1316049ef0>)

In [66]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 40000)

# Sample generation

this section is only used to generate examples. you can skip it if you just want to understand how the training works

In [67]:
# variation to https://github.com/ryankiros/skip-thoughts/blob/master/decoding/search.py
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in xrange(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [56]:
#!pip install python-Levenshtein

    100% |████████████████████████████████| 51kB 4.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for python-Levenshtein ... - \ | / done
  Stored in directory: /home/carnd/.cache/pip/wheels/c0/83/e9/b2cc2876e175d04091caf4e9f5de564ff2503b1f1885e7c3ba
Successfully built python-Levenshtein


In [68]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = map(len, samples)
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [69]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [70]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [71]:
import sys
import Levenshtein

def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
        codes.append(code)

In [58]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: Corrie^ live exclusive:^ Kevin,^ Tim,^ Sally plot injects^ ‘humour’^
DESC: Watch this exclusive interview with Michael Le Vell^ and Brooke Vincent talking to stv.tv^ about the Tim/Sally/Kevin^ ‘love^ triangle’^ storyline that will inject some light-hearted^
HEADS:


IndexError: index 0 is out of bounds for axis 1 with size 0

# Data generator

Data generator generates batches of inputs and outputs/labels for training. The inputs are each made from two parts. The first maxlend words are the original description, followed by `eos` followed by the headline which we want to predict, except for the last word in the headline which is always `eos` and then `empty` padding until `maxlen` words.

For each, input, the output is the headline words (without the start `eos` but with the ending `eos`) padded with `empty` words up to `maxlenh` words. The output is also expanded to be y-hot encoding of each word.

To be more realistic, the second part of the input should be the result of generation and not the original headline.
Instead we will flip just `nflips` words to be from the generator, but even this is too hard and instead
implement flipping in a naive way (which consumes less time.) Using the full input (description + eos + headline) generate predictions for outputs. For nflips random words from the output, replace the original word with the word with highest probability from the prediction.

In [72]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        #flips = sorted(random.sample(xrange(maxlend+1,maxlen), nflips))
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print(b,)
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]),)
            x_out[b,input_idx] = w
        if debug and b < debug:
            print()
    return x_out

In [73]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [74]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        #new_seed = random.randint(0, sys.maxint)
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [75]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 40000), 2)

In [76]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [77]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L:
Ventas,
Inc.
Announces
Quarterly
Dividend
of
<0>^
(VTR)
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Ventas,
Inc.
Announces
Quarterly
Dividend
of
<0>^
(VTR)
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
analysts
have
rated
the
stock
with
a
hold
rating
and
five
have
given
a
buy
rating
to
the
stock.
The
stock
currently
has
a
consensus

L:
Gaffaney
headed
to
Spain
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Gaffaney
headed
to
Spain
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
read
and
<3>^
At
<0>^
and
out
of
The
Snowden
School
and
McKenna
McKenna
College
<2>^
in
Business
Administration
-
Marketing),
he
averaged
<1>^
points,

L:
1
<0>^
2
injured
In
Sacramento
City
College
shooting
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
1
<0>^
2
injured
In
Sacramento
City
College
shooting
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
say
three
people
have
been
shot
and
one
of
them
has
died
near
a
baseball
field
on
the
campus
of
Sacramento
City
<1>^
Police
say

L:
adidas
Tubular
X
Arrives
In
Three
New
Colorways
~
_
_
_
_
_
_
_
_


test fliping

In [78]:
test_gen(gen(X_train, Y_train, nflips=6, model=model, debug=False, batch_size=batch_size))

L:
Ventas,
Inc.
Announces
Quarterly
Dividend
of
<0>^
(VTR)
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Ventas,
Inc.
Announces
Quarterly
Dividend
of
explains
(VTR)
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
analysts
have
rated
the
stock
with
a
hold
rating
and
five
have
given
a
buy
rating
to
the
stock.
The
stock
currently
has
a
consensus

L:
Gaffaney
headed
to
Spain
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Gaffaney
headed
to
mental
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
read
and
<3>^
At
<0>^
and
out
of
The
Snowden
School
and
McKenna
McKenna
College
<2>^
in
Business
Administration
-
Marketing),
he
averaged
<1>^
points,

L:
1
<0>^
2
injured
In
Sacramento
City
College
shooting
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
1
non-
2
injured
In
Sacramento
City
College
shooting
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
say
three
people
have
been
shot
and
one
of
them
has
died
near
a
baseball
field
on
the
campus
of
Sacramento
City
<1>^
Police
say

L:
adidas
Tubular
X
Arrives
In
Three
New
Colorways
~
_
_
_
_
_
_

In [79]:
valgen = gen(X_test, Y_test,nb_batches=3, batch_size=batch_size)

check that valgen repeats itself after nb_batches

In [80]:
for i in range(4):
    test_gen(valgen, n=1)

L:
In
Coimbatore
Today
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
In
Coimbatore
Today
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
City
Tower,
<1>^
10
a.m.
Vijaya
<3>^
Vattam:
<0>^
Sidhapudur
Government
High
School,
10
a.m.
Dr.
N.G.P
Arts
and
Science
<2>^
Parent
Teacher
Association
meeting,

L:
Hunt
for
3
suspects
in
Illinois
officer's
death
continues
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Hunt
for
3
suspects
in
Illinois
officer's
death
continues
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
of
a
popular
veteran
police
officer
in
a
small
northern
Illinois
community.
Lake
County
Sheriff's
Office
spokesman
Sgt.
Christopher
Covelli
said
hundreds
of
officers
were

L:
Butter
“Margarine
Fingers”
Ad
<0>^
~
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

H:
~
Butter
“Margarine
Fingers”
Ad
<2>^
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_

D:
a
<1>^
expression,
indicating
that
<3>^
fingers”
does
not
sound
at
all
<0>^
Then
they
decide
together
to
use
butter
in
the
dish
they
are

L:
In
Coimbatore
Today
~
_
_
_
_
_
_


# Train

In [81]:
history = {}

In [82]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)

In [84]:
r = next(traingen)
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 40000), 2)

In [85]:
for iteration in range(500):
    print('Iteration', iteration)
    h = model.fit_generator(traingen, samples_per_epoch=nb_train_samples,
                        nb_epoch=1, validation_data=valgen, nb_val_samples=nb_val_samples
                           )
    #for k,v in h.history.iteritems():
    for k,v in h.history.iteritems():
        history[k] = history.get(k,[]) + v
    with open('data/%s.history.pkl'%FN,'wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('data/%s.hdf5'%FN, overwrite=True)
    gensamples(batch_size=batch_size)

Iteration 0
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[64,25,40000]
	 [[Node: gradients/mul_1_grad/mul_1 = Mul[T=DT_FLOAT, _class=["loc:@mul_1"], _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_activation_1_target_0/_241, gradients/Sum_5_grad/Tile)]]
	 [[Node: gradients/while_1/Select_grad/Select/_601 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_5969_gradients/while_1/Select_grad/Select", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](^_cloopgradients/while_1/add_7_grad/BroadcastGradientArgs/StackPop/_192)]]

Caused by op 'gradients/mul_1_grad/mul_1', defined at:
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/dl/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carn